In [ ]:
#import findspark
#findspark.init()
import pyspark
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as func
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from sklearn.model_selection import KFold

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Assignment 1 Task2") \
    .getOrCreate()

sc = spark.sparkContext


In [ ]:
lines = spark.read.text("ml-25m/ratings.csv").rdd
parts = lines.map(lambda row: row.value.split(","))

header = parts.first()
parts = parts.filter(lambda line: line != header)

In [ ]:
for line in parts.take(5):
    print(line)

In [4]:
ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]),rating=float(p[2]), timestamp=int(p[3])))
ratings = spark.createDataFrame(ratingsRDD)

In [5]:
(fold_0, fold_1, fold_2) = ratings.randomSplit([1.0, 1.0, 1.0]).cache()

data_list = [fold_0,fold_1,fold_2]
test_list = [fold_2,fold_0,fold_1]

In [6]:
als_1 = ALS(rank=5,maxIter=10, regParam=0.1, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
als_2 = ALS(rank=40,maxIter=100, regParam=0.05, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")
als_3 = ALS(rank=80,maxIter=250, regParam=1.5, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

In [7]:
rmse_evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
mae_evaluator = RegressionEvaluator(metricName="mae", labelCol="rating",predictionCol="prediction")

In [8]:
def cross_validate(als_version):
    rmse_list =[]
    mae_list =[]
    fold_j = 0
    for fold_i in range(len(data_list)):
        if fold_j < len(data_list)-1:
            fold_j+= 1
        else:
            fold_j = 0
        first_train = data_list[fold_i].union(data_list[fold_j])
        model = als_version.fit(first_train)
        predictions = model.transform(test_list[fold_i])
        
        rmse = rmse_evaluator.evaluate(predictions)
        mae = mae_evaluator.evaluate(predictions)
        
        rmse_list.append(rmse)
        mae_list.append(mae)
    
    return rmse_list, mae_list

In [9]:
rmse_als_1, mae_als_1 = cross_validate(als_1)

In [11]:
rmse_als_2, mae_als_2 = cross_validate(als_2)

Py4JJavaError: An error occurred while calling o92.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 406.0 failed 1 times, most recent failure: Lost task 0.0 in stage 406.0 (TID 4005, localhost, executor driver): java.lang.StackOverflowError
	at java.io.ObjectInputStream$BlockDataInputStream.readInt(Unknown Source)
	at java.io.ObjectInputStream.readHandle(Unknown Source)
	at java.io.ObjectInputStream.readClassDesc(Unknown Source)
	at java.io.ObjectInputStream.readOrdinaryObject(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.defaultReadFields(Unknown Source)
	at java.io.ObjectInputStream.readSerialData(Unknown Source)
	at java.io.ObjectInputStream.readOrdinaryObject(Unknown Source)
	at java.io.ObjectInputStream.readObject0(Unknown Source)
	at java.io.ObjectInputStream.readObject(Unknown Source)
	at scala.collection.immutable.List$SerializationProxy.readObject(List.scala:479)
	at 

In [12]:
rmse_als_3, mae_als_3 = cross_validate(als_3)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:10823)
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1145, in send_command
    self.socket.sendall(command.encode("utf-8"))
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 985, in send_command
    response = connection.send_command(command)
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1149, in send_command
    "Error while sending", e, proto.ERROR_ON_SEND)
py4j.protocol.Py4JNetworkError: Error while sending

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-package

Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:10823)

In [10]:
rmse_als_1

[0.8138968855561215, 0.8136773416579973, 0.8135812944409176]

In [11]:
mae_als_1

[0.6295640139821601, 0.6286027257023543, 0.6294547920581957]